<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parse-program-info-and-visit-timeline-webpages" data-toc-modified-id="Parse-program-info-and-visit-timeline-webpages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Parse program info and visit timeline webpages</a></span><ul class="toc-item"><li><span><a href="#Prog-IDs-from-the-web" data-toc-modified-id="Prog-IDs-from-the-web-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Prog IDs from the web</a></span></li><li><span><a href="#Cal-programs" data-toc-modified-id="Cal-programs-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cal programs</a></span></li><li><span><a href="#Scrape-visit-scheduling" data-toc-modified-id="Scrape-visit-scheduling-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Scrape visit scheduling</a></span></li><li><span><a href="#Write-to-table" data-toc-modified-id="Write-to-table-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Write to table</a></span></li></ul></li></ul></div>

# Parse program info and visit timeline webpages



In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import requests

In [4]:
# https://stackoverflow.com/questions/24124643/parse-xml-from-url-into-python-object
import traceback
import urllib3
import xmltodict

def getxml(url="https://yoursite/your.xml"):
    http = urllib3.PoolManager()

    response = http.request('GET', url)
    try:
        data = xmltodict.parse(response.data)
    except:
        print("Failed to parse xml from response (%s)" % traceback.format_exc())
        data = {}
        
    return data

def visit_xml(proposal_id=1324):
    url = f"https://www.stsci.edu/cgi-bin/get-visit-status?id={proposal_id}&markupFormat=xml&observatory=JWST"
    data = getxml(url=url)
    if 'visitStatusReport' in data:
        data = data['visitStatusReport']
        
    return data

def prop_html(proposal_id=1324):
    from bs4 import BeautifulSoup
    vgm_url = f"https://www.stsci.edu/cgi-bin/get-proposal-info?id={proposal_id}&observatory=JWST"
    html_text = requests.get(vgm_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    return soup

def program_info(proposal_id=1324):
    
    soup = prop_html(proposal_id=proposal_id)
    meta = {'proposal_id':proposal_id}
    meta['raw'] = soup
    
    if 1:
        ps = soup.findAll('p')
        meta['pi'] = ps[0].contents[1].strip()
        meta['title'] = ps[1].contents[1].strip()
        meta['cycle'] = int(ps[1].contents[5].strip())
        meta['allocation'] = float(ps[1].contents[9].strip().split()[0])
        meta['proptime'] = float(ps[1].contents[-1].strip().split()[0])
        meta['type'] = soup.findAll('h1')[0].contents[1].contents[0]
    else:
        meta['pi'] = 'x'
        meta['title'] = 'x'
        meta['cycle'] = 0
        meta['allocation'] = 0
        meta['proptime'] = 0.
        meta['type'] = 'x'
        
    visits = visit_xml(proposal_id)
    #for k in ['visit']: #visits:
    #    meta[k] = visits[k]
    if isinstance(visits['visit'], list):
        meta['visit'] = visits['visit']
    else:
        meta['visit'] = [visits['visit']]
        
    return meta

    #return soup

#visit = visit_xml(proposal_id=1324)
meta = program_info(proposal_id=1567)

In [27]:
from grizli import utils
import astropy.units as u
import astropy.time
import datetime

def show_window(v, meta):
    
    row = [meta[k] for k in ['type','proposal_id','title', 'pi', 'cycle', 'allocation', 'proptime']]
    row += [v[k] for k in ['@observation', '@visit', 'target', 'configuration', 'hours']]

    archived = False
    
    if v['status'] == 'Archived':
        t0 = datetime.datetime.strptime(v['startTime'], "%b %d, %Y %H:%M:%S")
        yd0 = '.'.join(astropy.time.Time(t0, format='datetime').yday.split(':')[:2])
        t1 = datetime.datetime.strptime(v['endTime'], "%b %d, %Y %H:%M:%S")
        yd1 = '.'.join(astropy.time.Time(t0, format='datetime').yday.split(':')[:2])
        dates = (yd0, yd1)
        archived = True
    elif v['status'] == 'Scheduled':
        
        t0 = datetime.datetime.strptime(v['startTime'], "%b %d, %Y %H:%M:%S")
        yd0 = '.'.join(astropy.time.Time(t0, format='datetime').yday.split(':')[:2])
        t1 = datetime.datetime.strptime(v['endTime'], "%b %d, %Y %H:%M:%S")
        yd1 = '.'.join(astropy.time.Time(t0, format='datetime').yday.split(':')[:2])
        dates = (yd0, yd1)
        archived = 2
        
    elif 'planWindow' not in v:
        row.extend(['2029-01-01','-','-'])
        return row, None
    else:    
        w = v['planWindow']

        if '(2' in w:
            dates = ('2'+w.split('(2')[1].strip()).strip(')').split(' - ')
        else:
            row.extend(['2029-01-01','-','-'])
            return row, None
    
    inst = v['configuration'].strip().split()[0].lower()

    colors = {'niriss':utils.MPL_COLORS['b'], 
              'nirspec':utils.MPL_COLORS['orange'], 
              'miri':utils.MPL_COLORS['r'],
              'nircam':utils.MPL_COLORS['g']}
    
    if inst not in colors:
        colors[inst] = '0.5'
        
    fig, ax = plt.subplots(1,1,figsize=(8,0.4))

    wlim = astropy.time.Time(['2022:150','2024:300'], format='yday')
    # ax.plot_date(wlim.plot_date, [0,0], color='w', lw=5, linestyle='-')

    if archived == 1:
        fmt = 'x'
        ofmt = 'x'
    elif archived == 2:
        fmt = 'v'
        ofmt = 'v'        
    else:
        fmt = 's'
        ofmt = 'o'

    wdate = astropy.time.Time([d.replace('.',':') for d in dates], format='yday')
    ax.plot_date(wdate.plot_date, [0,0], color=colors[inst], lw=2, fmt=fmt, linestyle='-')

    prop = meta['proptime']
    #prop = 6

    if prop > 0:
        pub = wdate + prop/12.*u.year
        ax.plot_date(wdate.plot_date, [0,0], fmt=ofmt, zorder=100, color='w', markersize=4)
        ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt, 
                     alpha=0.5)

    wgrid = astropy.time.Time(['2022-07-01', '2022-10-01', 
                               '2023-01-01', '2023-04-01', '2023-07-01', '2023-10-01', 
                               '2024-01-01', '2024-04-01', '2024-07-01', '2024-10-01'])

    ax.vlines(wgrid.plot_date[[2,6]], -1, 1, color='k', alpha=0.2)
    ax.set_ylim(-0.2, 0.2)
    #ax.axis('off')

    #ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    #ax.spines['left'].set_visible(False)

    ax.set_xticks(wgrid.plot_date)
    ax.set_xlim(*wlim.plot_date)

    ax.tick_params(length=0, which='both')

    ax.get_yaxis().set_visible(False)

    ax.set_xticklabels([])
    ax.set_yticklabels([])

    ax.grid()
    
    iso = [w.iso.split()[0] for w in wdate]
    # dates, meta['visit'][0], meta

    row += iso

    timeline = f"jwst_{meta['proposal_id']}_{v['@observation']}_{v['@visit']}"
    
    row.append(f'<img src="{timeline}.png" />')
    
    fig.savefig(timeline+'.png')

    return row, fig

In [28]:
progs = [1837, # primer
         1345, # ceers
         1433, # coe
         1727, # cosmos
         2079, # ngdeep
         2426, # bagley
         2659, # beasts bubbles
         2561, # uncover
         1914, # aurora
         1895, # fresco
         1567, # seiji
         1324, # glass
         1208, # canucs
         1180, # eisenstein, gs1
         1286, # gs2
         1287, # gs3
         1181, # gn1
         1210, # ferruit
         1211, # ferruit NRS GTO
         1213, 
         1214, 
         1215, 
         1216,
         1217, 
         1262, 
         1263, # colina
         1284, # colina 2
         1264, # colina 3
         1283, # miri udf
         1243, # lilly
         1176, # windhorst
         1207, # rieke
         2282, # earendel
         1869, # LyC22
         1626, # stefanon
         1657, # harikane
         1671, # maseda
         1740, # harikane 2
         1747, # roberts-borsani
         1758, # finkelstein
         1791, # spilker
         1810, # belli
         1933, # mathee
         1963, # williams udf
         2110, # kriek
         2136, # tucker
         2279, # naidu
         2285, # carnall
         2345, # newman
         2362, # marsan
         2478, # stark
         2484, # mirka
         2516, # hodge
         2555, # sunburst
         2565, # glazebrook, 
         1908, # vanzella
         1764, # fan agn
         1964, # overzier
         1328, # armus
         1355, # rigby
         1199, # stiavelli
         2123, # kassin
         2130, # lee
         2198, # barrufet
         2234, # banados
         2321, # best
         2566, 
         2674,
         1871,
         2078, 
         1678, # ashby
         2107, 
         1717, 
         1554,
         2593, # strom
        ]
#
progs = [1345]


## Prog IDs from the web

In [31]:
from bs4 import BeautifulSoup
import requests

progs = []

for url in ["https://www.stsci.edu/jwst/science-execution/approved-ers-programs", 
            "https://www.stsci.edu/jwst/science-execution/approved-programs/cycle-1-gto", 
            "https://www.stsci.edu/jwst/science-execution/approved-programs/cycle-1-go"]:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    tabs = soup.findAll('tbody')
    for tab in tabs:
        for link in tab.findAll('a'):
            try:
                prop = int(link.getText())
                progs.append(prop)
            except:
                continue

len(progs)

419

## Cal programs

In [32]:
cprogs = '1517-1533,1453,1475-1483,1550,1497-1499,1501,1503,1504,1506-1510,1512,1514,1515,1516,1484-1489,1491-1496,1534-1539'
calibs = []
for p in cprogs.split(','):
    if '-' in p:
        spl = p.split('-')
        calibs += [i for i in range(int(spl[0]), int(spl[1])+1)]
    else:
        calibs.append(int(p))

progs += calibs
        

In [33]:
len(progs)

480

In [35]:
rows = []
done = []
failed = []

len(progs)
res = {}

## Scrape visit scheduling

In [36]:
os.chdir('/Users/gbrammer/Research/JWST/LRP')
names = ['type','proposal_id','title', 'pi', 'cycle', 'allocation', 'proptime',
         'observation', 'visit', 'target', 'configuration', 'hours', 
         'win_start', 'win_end', 'timeline']

# progs = [1963]

for i, prog in enumerate(progs):
    print(i, prog)
    if prog in done:
        continue
    
    try:
        m = program_info(proposal_id=prog)
    except IndexError:
        failed.append(prog)
        print('Fail')
        continue
    except TypeError:
        failed.append(prog)
        continue
        
    done.append(prog)
    res[prog] = m
    
    for v in m['visit']:
        try:
            row, fig = show_window(v, m)  
        except KeyError:
            continue
            
        rows.append(row)
    
        plt.close('all')

0 1324
1 1328
2 1345
3 1355
4 1335
5 1364
6 1366
7 1386
8 1373
9 1288
10 1309
11 1349
12 1334
13 1189


/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linesty

14 1202


/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linesty

15 1209


...

/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linesty

479 1539


/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linesty

/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linesty

/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,
/var/folders/tv/ycbhjzmd34j5ntfdldnxf5xr0000gn/T/ipykernel_37072/1030065652.py:75: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "s-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(pub.plot_date, [0,0], color=colors[inst], lw=2, linestyle=':', fmt=fmt,


In [37]:
failed

[1195,
 1255,
 1977,
 2232,
 2509,
 1721,
 1883,
 2354,
 2410,
 2687,
 2608,
 1585,
 1598,
 2037,
 2524,
 2537,
 1557,
 2074,
 2352,
 2108,
 2446,
 2554]

## Write to table

In [54]:
tab = utils.GTable(rows=rows, names=names)
# tab['timeline'] = [f'<img src="{t}.png" />' for t in tab['timeline']]
tab['prop'] = tab['proposal_id']

tab['proposal_id'] = [f'<a href="https://www.stsci.edu/cgi-bin/get-proposal-info?id={p}&observatory=JWST">{p}</a>'
                      for p, t in zip(tab['proposal_id'], tab['type'])]

# tab['pi'] = [p.strip().split()[-1] for p in tab['pi']]

prop = []
for p in tab['proptime']:
    if p > 0:
        prop.append(f'<p style="color:red;"> {p:.0f} </p>')
    else:
        prop.append(f'<p style="color:green;"> {p:.0f} </p>')

tab['proptime'] = prop

targ = []
for t in tab['target']:
    if isinstance(t, list):
        targ.append('; '.join(t))
    else:
        targ.append(t)
        
tab['target'] = targ

tab.rename_column('proposal_id','ProgID')
tab.rename_column('proptime','Prop')
tab.rename_column('pi','PI')
tab.rename_column('title','Full_Proposal_Title')
tab.rename_column('allocation', 'Total')
tab.rename_column('hours', 'Hours')

tab['Prop'].description = 'Proprietary time, months'
tab['Hours'].description = 'Visit duration'
tab['Total'].description = 'Total program allocation, hours'
tab['win_start'].description = 'Beginning of LRP scheduling window'
tab['win_end'].description = 'End of LRP scheduling window'

tab.rename_column('observation','Obs')

output = 'full_timeline_v2.html'
output = 'full_timeline_v3.html'

tab.write(output.replace('.html','.csv'), overwrite=True)

cols = ['type', 'ProgID', 'Full_Proposal_Title', 'PI', 'cycle',
        'Total', 'Prop', 'Obs', 'visit', 'target', 'configuration',
        'Hours', 'win_start', 'win_end', 'timeline']

tab[cols].write_sortable_html(output, use_json=False, 
                        localhost=False, max_lines=10000, filter_columns=['Total', 'Prop','Hours','win_start','win_end'])

with open(output) as fp:
    lines = fp.readlines()
#
lines.insert(-3, f'<p style="font-size:x-small;"> Generated {astropy.time.Time.now().iso} by {os.getlogin()} with <a href="https://github.com/gbrammer/jwst_EERS/blob/main/lrp-calendar.ipynb">lrp-calendar.ipynb</a> </p>\n')
with open(output,'w') as fp:
    fp.writelines(lines)

In [55]:
print(tab.colnames)

['type', 'ProgID', 'Full_Proposal_Title', 'PI', 'cycle', 'Total', 'Prop', 'Obs', 'visit', 'target', 'configuration', 'Hours', 'win_start', 'win_end', 'timeline', 'prop']


In [56]:
os.chdir('/Users/gbrammer/Research/JWST/LRP')

xtab = utils.read_catalog(output.replace('.html','.csv'))

xtab[:10]


type,ProgID,Full_Proposal_Title,PI,cycle,Total,Prop,Obs,visit,target,configuration,Hours,win_start,win_end,timeline,prop
str11,str91,str148,str28,int64,float64,str31,int64,int64,str82,str42,float64,str10,str10,str33,int64
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1324&observatory=JWST"">1324</a>",Through the looking GLASS: a JWST exploration of galaxy formation and evolution from cosmic dawn to present day,Tommaso Treu,1,35.5,"<p style=""color:green;""> 0 </p>",1,1,ABELL2744,NIRISS Wide Field Slitless Spectroscopy,14.92,2029-01-01,-,-,1324
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1324&observatory=JWST"">1324</a>",Through the looking GLASS: a JWST exploration of galaxy formation and evolution from cosmic dawn to present day,Tommaso Treu,1,35.5,"<p style=""color:green;""> 0 </p>",6,1,ABEL2744CLS-V12,NIRSpec MultiObject Spectroscopy,20.23,2029-01-01,-,-,1324
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1328&observatory=JWST"">1328</a>",A JWST Study of the Starburst-AGN Connection in Merging LIRGs,Lee Armus,1,38.5,"<p style=""color:green;""> 0 </p>",17,1,NGC-7469,MIRI Imaging,0.66,2022-06-22,2022-06-22,"<img src=""jwst_1328_17_1.png"" />",1328
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1328&observatory=JWST"">1328</a>",A JWST Study of the Starburst-AGN Connection in Merging LIRGs,Lee Armus,1,38.5,"<p style=""color:green;""> 0 </p>",15,1,NGC-7469-MRS,MIRI Medium Resolution Spectroscopy,1.71,2022-07-01,2022-07-30,"<img src=""jwst_1328_15_1.png"" />",1328
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1328&observatory=JWST"">1328</a>",A JWST Study of the Starburst-AGN Connection in Merging LIRGs,Lee Armus,1,38.5,"<p style=""color:green;""> 0 </p>",16,1,NGC-7469-BK,MIRI Medium Resolution Spectroscopy,1.15,2022-07-01,2022-07-30,"<img src=""jwst_1328_16_1.png"" />",1328
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1328&observatory=JWST"">1328</a>",A JWST Study of the Starburst-AGN Connection in Merging LIRGs,Lee Armus,1,38.5,"<p style=""color:green;""> 0 </p>",18,1,NGC-7469,NIRSpec IFU Spectroscopy,2.43,2022-07-01,2022-07-30,"<img src=""jwst_1328_18_1.png"" />",1328
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1328&observatory=JWST"">1328</a>",A JWST Study of the Starburst-AGN Connection in Merging LIRGs,Lee Armus,1,38.5,"<p style=""color:green;""> 0 </p>",19,1,NGC-7469,NIRCam Imaging,0.8,2022-07-01,2022-07-30,"<img src=""jwst_1328_19_1.png"" />",1328
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1328&observatory=JWST"">1328</a>",A JWST Study of the Starburst-AGN Connection in Merging LIRGs,Lee Armus,1,38.5,"<p style=""color:green;""> 0 </p>",20,1,VV114-NUC-MRS,MIRI Medium Resolution Spectroscopy,1.35,2022-07-01,2022-08-21,"<img src=""jwst_1328_20_1.png"" />",1328
ERS,"<a href=""https://www.stsci.edu/cgi-bin/get-proposal-info?id=1328&observatory=JWST"">1328</a>",A JWST Study of the Starburst-AGN Connection in Merging LIRGs,Lee Armus,1,38.5,"<p style=""color:green;""> 0 </p>",21,1,VV114-BK,MIRI Medium Resolution Spectroscopy,1.35,2022-07-01,2022-08-21,"<img src=""jwst_1328_21_1.png"" />",1328


In [57]:
# Fetch APT files for targets
apt_exec = f'/Applications/APT\ 2022.1.1/bin/apt'
for prog in progs:
    print(prog)
        
    if not os.path.exists(f'APT_Files/{prog}.aptx'):
        os.system(f'wget -O APT_Files/{prog}.aptx "https://www.stsci.edu/jwst/phase2-public/{prog}.aptx"')
    
    if not os.path.exists(f'APT_Files/{prog}.pointing'):
        os.system(f'{apt_exec} APT_Files/{prog}.aptx -nogui -export times,xml,pointing,msatargets,csv')
        os.system(f'rm APT_Files/{prog}*backup')
    
    if not os.path.exists(f'APT_Files/{prog}.csv'):
        os.system(f'{apt_exec} APT_Files/{prog}.aptx -nogui -export csv')
        os.system(f'rm APT_Files/{prog}*backup')
        

1324
1328
1345
1355
1335
1364
1366
1386
1373
1288
1309
1349
1334
1189
1202
1209
1228
1229
1230
1256
1413
1414
1199
1208
1183
1192
1206
1282
1294
1411
1180
1181
1207
1210
1211
1213
1214
1215
1216
1217
1262
1263
1283
1284
1286
1287
1177
1184
1185
1188
1193
1194
1195
1200
1201
1224
1241
1270
1274
1275
1276
1277
1278
1279
1280
1281
1292
1312
1331
1353
1412
1198
1205
1218
1219
1220
1222
1243
1264
1179
1186
1187
1235
1236
1257
1258
1282
1290
1293
1191
1231
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1271
1272
1273
1415
1182
1190
1232
1237
1238
1239
1240
1242
1306
1176
1178
1204
1223
1225
1226
1227
1233
1234
1260
1265
1266
1267
1268
1269
1304
1305
1549
1556
1563
1584
1618
1633
1640
1647
1668
1676
1729
1743
1751
1759
1803
1843
1846
1902
1911
1935
1952
1958
1977
1981
2001
2008
2021
2025
2044
2053
2055
2062
2084
2113
2149
2153
2158
2159
2232
2243
2260
2278
2297
2304
2311
2319
2334
2347
2358
2372
2420
2437
2454
2487
2488
2498
2507
2508
2509
2512
2538
2540
2562
2571
2589
2594
2627


In [58]:
# Read targets
colors = {'NIS':utils.MPL_COLORS['b'], 
          'NRS':utils.MPL_COLORS['orange'], 
          'MIR':utils.MPL_COLORS['r'],
          'NRC':utils.MPL_COLORS['g'], 
          'FGS':'olive'}

def read_targets(prog=1355, make_figure=False, file=None):
    if file is None:
        csv = utils.read_catalog(f'APT_Files/{prog}.csv')
    else:
        csv = utils.read_catalog(file)
        prog = int(os.path.basename(file).split('.csv')[0])
        
    #csv['visit'] = [int(v[4:8]) for v in csv['Visit ID']]
    csv['Obs'] = ((csv['Visit ID'] % 1000000 - 1)/1000).astype(int)
    
    #un = utils.Unique(csv['Target'], verbose=False)
    un = utils.Unique(csv['Obs'], verbose=False)
    rows = []
    for v, sel in un:
        
        s_region = ' '.join(csv['s_region'][sel].tolist())
        sr = utils.SRegion(s_region)
        
        #apers = [a[:3] for a in csv['Aperture'][sel]]
        
        rd = np.squeeze(sr.union(as_polygon=True).centroid.xy)
        row = [prog, csv['Obs'][sel][0], csv['Target'][sel][0], *rd]
        
        rows.append(row)
        
        if make_figure:
            fig, ax = plt.subplots(1,1,figsize=(2,2))
            ax.scatter(csv['RA Center Rot'][sel], csv['Dec Center Rot'][sel], marker='.', alpha=0.0, color='r', zorder=1)
            for i, p in enumerate(sr.patch(alpha=0.1)):
                p.set_facecolor(colors[apers[i]])
                ax.add_patch(p)

            ax.set_xticks([]) #np.mean(ax.get_xlim())])
            ax.set_yticks([]) #np.mean(ax.get_ylim())])

            #ax.set_yticklabels([f'{t:.5f}' for t in ax.get_yticks()], rotation = 90)

            ax.set_aspect(1./np.cos(csv['Dec Center Rot'][sel][0]/180*np.pi))
            ax.grid()
            ax.set_xlim(*ax.get_xlim()[::-1])
            fig.tight_layout(pad=0.)
    
    targ = utils.GTable(names=['ProgID','Obs','target','ra','dec'], rows=rows)
    return targ

csv = read_targets(prog=1837, make_figure=False)
csv

ProgID,Obs,target,ra,dec
int64,int64,str13,float64,float64
1837,1,UDS-1,34.36737748727084,-5.130058755300024
1837,2,UDS-2,34.37757164344824,-5.270052401505208
1837,3,COSMOS-1,150.18034260248535,2.4004628123267238
1837,4,COSMOS-2,150.07166782403294,2.2637826190663017
1837,8,UDS-1-TILE-13,34.26771262520945,-5.151317724130026
1837,9,UDS-1-TILE-23,34.262423179900786,-5.131442405795684
1837,14,UDS-2-TILE-6,34.3504344306357,-5.29091009626964
1837,21,UDS-1-TILE-9,34.43085772638597,-5.150585694785755
1837,22,UDS-2-TILE-22,34.443040206758695,-5.27059088424836


In [59]:
import glob
from tqdm import tqdm

all_files = glob.glob('APT_Files/????.csv')
all_files.sort()
targs = []
targets = []

for file in tqdm(all_files):
    try:
        _targ = read_targets(file=file, make_figure=False)
        targets += _targ['target'].tolist()
        _targ.remove_column('target')
        targs.append(_targ)
        
    except ValueError:
        print('Error: ', file)


 35%|█████████████████████████████████████████████▏                                                                                   | 168/479 [00:02<00:02, 115.59it/s]

Error:  APT_Files/1453.csv
Error:  APT_Files/1477.csv
Error:  APT_Files/1479.csv
Error:  APT_Files/1480.csv
Error:  APT_Files/1481.csv
Error:  APT_Files/1483.csv
Error:  APT_Files/1484.csv
Error:  APT_Files/1485.csv
Error:  APT_Files/1486.csv
Error:  APT_Files/1488.csv
Error:  APT_Files/1489.csv
Error:  APT_Files/1491.csv
Error:  APT_Files/1494.csv
Error:  APT_Files/1495.csv
Error:  APT_Files/1496.csv
Error:  APT_Files/1497.csv
Error:  APT_Files/1498.csv
Error:  APT_Files/1499.csv
Error:  APT_Files/1501.csv
Error:  APT_Files/1503.csv
Error:  APT_Files/1506.csv
Error:  APT_Files/1507.csv
Error:  APT_Files/1510.csv
Error:  APT_Files/1512.csv
Error:  APT_Files/1515.csv
Error:  APT_Files/1517.csv
Error:  APT_Files/1518.csv
Error:  APT_Files/1519.csv
Error:  APT_Files/1520.csv


 38%|████████████████████████████████████████████████▋                                                                                | 181/479 [00:02<00:02, 112.32it/s]

Error:  APT_Files/1532.csv
Error:  APT_Files/1534.csv
Error:  APT_Files/1535.csv


 43%|███████████████████████████████████████████████████████▎                                                                          | 204/479 [00:03<00:03, 69.06it/s]

...

Error:  APT_Files/2446.csv
Error:  APT_Files/2509.csv
Error:  APT_Files/2524.csv
Error:  APT_Files/2537.csv


 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊     | 460/479 [00:07<00:00, 93.80it/s]

Error:  APT_Files/2554.csv
Error:  APT_Files/2608.csv


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 479/479 [00:07<00:00, 65.69it/s]

Error:  APT_Files/2687.csv


In [60]:
all_targs = astropy.table.vstack(targs)
all_targs['target'] = targets

url = '<a href="https://www.legacysurvey.org/viewer?ra={0:.6f}&dec={1:.6f}&layer=ls-dr9&zoom=14"> sky </a>'

all_targs['sky'] = [url.format(r, d) for r, d in zip(all_targs['ra'], all_targs['dec'])]

tab['ra'] = -1.
tab['dec'] = -1.

skies = ['---']*len(tab)
if 'sky' in tab.colnames:
    tab.remove_column('sky')

for j, (p, o) in enumerate(zip(tab['prop'], tab['Obs'])):
    ix = all_targs['ProgID'] + all_targs['Obs']/1000. == p+float(o)/1000.
    aix = all_targs[ix]
    if ix.sum() == 0:
        continue
        
    tab['ra'][j] = aix['ra'][0]
    tab['dec'][j] = aix['dec'][0]
    skies[j] = aix['sky'][0]
    
tab['sky'] = skies

In [61]:
cols = ['type', 'ProgID', 'Full_Proposal_Title', 'PI', 'cycle',
        'Total', 'Prop', 'Obs', 'visit', 'target', 'ra','dec','sky','configuration',
        'Hours', 'win_start', 'win_end', 'timeline']

tab['ra'].format = '.6f'
tab['dec'].format = '.6f'
tab['ra'].description = 'Visit RA center, deg'
tab['dec'].description = 'Visit DEC center, deg'

tab[cols].write_sortable_html(output, use_json=True, 
                        localhost=False, max_lines=10000,
                              filter_columns=['Total', 'Prop','Hours','win_start','win_end','ra','dec'])

with open(output) as fp:
    lines = fp.readlines()
#
lines.insert(-3, f'<p style="font-size:x-small;"> Generated {astropy.time.Time.now().iso} by {os.getlogin()} with <a href="https://github.com/gbrammer/jwst_EERS/blob/main/lrp-calendar.ipynb">lrp-calendar.ipynb</a> </p>\n')
with open(output,'w') as fp:
    fp.writelines(lines)
    

In [62]:
csv.colnames

['ProgID', 'Obs', 'target', 'ra', 'dec']

In [48]:
sel = csv['dec'] > -100
s_region = ' '.join(csv['s_region'][sel].tolist())
sr = utils.SRegion(s_region)

apers = [a[:3] for a in csv['Aperture'][sel]]

if 1:
    fig, ax = plt.subplots(1,1,figsize=(2,2))
    ax.scatter(csv['RA Center Rot'][sel], csv['Dec Center Rot'][sel], marker='.', alpha=0., color='r', zorder=1)
    for i, p in enumerate(sr.patch(alpha=0.1)):
        p.set_facecolor(colors[apers[i]])
        ax.add_patch(p)

    ax.set_xticks([]) #np.mean(ax.get_xlim())])
    ax.set_yticks([]) #np.mean(ax.get_ylim())])

    #ax.set_yticklabels([f'{t:.5f}' for t in ax.get_yticks()], rotation = 90)

    ax.set_aspect(1./np.cos(csv['Dec Center Rot'][sel][0]/180*np.pi))
    ax.grid()
    ax.set_xlim(*ax.get_xlim()[::-1])
    fig.tight_layout(pad=0.)


KeyError: 'Dec Center Rot'

In [69]:
utils.Unique(csv['Visit ID'] % 1000000)

   N  value     
====  ==========
   8        1001
   8       98001
   8       99001
   8      100001
   8      101001
   8      102001
   8      103001
   8      104001
   8      105001
   8      106001
   8      107001
   8      108001
   8      109001
   8      110001
   8      111001
   8      112001
   8       97001
   8      113001
   8       96001
   8       94001
   8       79001
   8       80001
   8       81001
   8       82001
   8       83001
   8       84001
   8       85001
   8       86001
   8       87001
   8       88001
   8       89001
   8       90001
   8       91001
   8       92001
   8       93001
   8       95001
   8       78001
   8      114001
   8      116001
   8      136001
   8      137001
   8      138001
   8      139001
   8      140001
   8      141001
   8      142001
   8      143001
   8      144001
   8      145001
   8      146001
   8      147001
   8      148001
   8      149001
   8      150001
   8      135001
   8      115001
   8      1340